### rented movies = 747

In [15]:
!pip install lxml

    100% |████████████████████████████████| 7.1MB 180kB/s ta 0:00:01


In [22]:
from urllib.parse import urlencode
from lxml.html import parse
import pandas as pd
import numpy as np

In [18]:
from lxml import html
import requests

## find movie

In [53]:
def find_movie(film):
    try:
        url = "http://www.csfd.cz/hledat/" + '?' + urlencode({'q' : film})
        root = parse(url).getroot()
        page = requests.get(url)
        tree = html.fromstring(page.content)
        doc_movies = tree.xpath("//div[@id='search-films']/div[1]")[0]
        l = doc_movies.xpath("ul[1]/li")[0]
        href = l.xpath("a")[0].get("href")
    except:
       return None                                                         
    #print(l.xpath("a/img")[0].get("src"))
    #print(l.xpath("div/h3/a")[0].text)
    return href;

In [37]:
find_movie('kokot')

'/film/321575-kokot-a-perly/'

In [56]:
def get_movie_detail(film):
    
    nadpis = None,
    img = None
    genresnew = None
    score = None
    relatednew = None
    try:
        url = "http://www.csfd.cz" + film
        root = parse(url).getroot()
        page = requests.get(url)
        tree = html.fromstring(page.content)
        nadpis = tree.xpath("//h1")[0].text.strip()
        img = tree.xpath("//div[@id='profile']/div/div/img")[0].get("src")
        img = "http:" + img
        text = tree.xpath("//div[@id='profile']//p[@class='genre']")[0].text
        genresold = text.split("/")
        genresnew = []
        for x in genresold: 
            genresnew.append(x.strip())
        score = tree.xpath("//h2[@class='average']")[0].text
        relatedold = tree.xpath("//div[@class='ct-related related']/div/ul/li/a")
        relatednew = []
        for x in relatedold: 
            relatednew.append(x.get("href"))
    except:
        pass
    return {'title': nadpis, 'img': img, 'genres': genresnew, 'score': score, 'related': relatednew};

In [57]:
get_movie_detail("/film/1249-terminator/")

{'genres': ['Akční', 'Sci-Fi', 'Thriller'],
 'img': 'http://img.csfd.cz/files/images/film/posters/000/001/1251_697bf1.jpg?h180',
 'related': ['/film/263921-terminator-genisys/',
  '/film/227774-terminator-salvation/',
  '/film/268365-terminator-salvation-temny-pocatek/',
  '/film/230905-terminator-pribeh-sary-connorove/',
  '/film/15908-terminator-3-vzpoura-stroju/',
  '/film/42922-t2-3-d-battle-across-time/',
  '/film/362169-guns-n-roses-you-could-be-mine/',
  '/film/1248-terminator-2-den-zuctovani/'],
 'score': '87%',
 'title': 'Terminátor'}

In [58]:
def get_movie(film):
    movie_detail = None
    movie = find_movie(film)
    if movie is not None:
        movie_detail = get_movie_detail(movie)
    return movie_detail

In [59]:
get_movie('kokot')['title']

'Kokot a perly'

In [4]:
data = pd.read_csv('../query-hive-486.csv', names=['name', 'gender', 'cnt'])

In [85]:
sex = data.name.str.contains('Hustler|Bohem|Dorcel|Leo', regex=True)

### Sex movies ratio

In [86]:
np.round(np.sum(sex)/data.shape[0],2)*100

48.0

In [43]:
pd.Series(get_movie('kokot'))

genres           [Dokumentární, Krátkometrážní, Životopisný]
img        http:http://img.csfd.cz/assets/b2206/images/po...
related                                                   []
score                                                    74%
title                                          Kokot a perly
dtype: object

## Group recommender

In [77]:
%%time
movie_details = data.apply(lambda x : pd.Series(get_movie(x['name'])), axis=1)
all_data = pd.concat([data, movie_details], axis=1)
no_sex_data = all_data[~sex]

CPU times: user 29.9 s, sys: 1.86 s, total: 31.7 s
Wall time: 8min 58s


In [105]:
def recommand_for_gender(gender, limit=10):
    f = no_sex_data[no_sex_data.gender == gender]
    return f[['title', 'score', 'img']].sort_values(by='score', ascending=False)[:limit]

In [118]:
muzi = recommand_for_gender('M')
muzi

,title,score,img
110,National Geographic Explorer,90%,http://img.csfd.cz/files/images/film/posters/0...
151,Mŕtva nevesta Tima Burtona,84%,http://img.csfd.cz/files/images/film/posters/1...
87,American Experience,84%,http://img.csfd.cz/files/images/film/posters/0...
146,Sully,83%,http://img.csfd.cz/files/images/film/posters/1...
64,Vlk z Wall Street,83%,http://img.csfd.cz/files/images/film/posters/1...
63,Izba,82%,http://img.csfd.cz/files/images/film/posters/1...
68,"South Park: Bigger, Longer & Uncut",81%,http://img.csfd.cz/files/images/film/posters/0...
177,"Bež, chlapče, bež",80%,http://img.csfd.cz/files/images/film/posters/1...
86,"Muži, ktorí nenávidia ženy",79%,http://img.csfd.cz/files/images/film/posters/1...
72,Účtovník,79%,http://img.csfd.cz/files/images/film/posters/1...


In [117]:
zeny = recommand_for_gender('F')
zeny

,title,score,img
207,National Geographic Explorer,90%,http://img.csfd.cz/files/images/film/posters/0...
284,Návrat Temného rytiera,86%,http://img.csfd.cz/files/images/film/posters/0...
237,Jindřich IV.,85%,http://img.csfd.cz/files/images/film/posters/0...
228,American Experience,84%,http://img.csfd.cz/files/images/film/posters/0...
269,Sully,83%,http://img.csfd.cz/files/images/film/posters/1...
240,Náš vodca,82%,http://img.csfd.cz/files/images/film/posters/1...
229,... a rána sú tu tiché,78%,http://img.csfd.cz/files/images/film/posters/1...
298,V zajatí démonov 2,78%,http://img.csfd.cz/files/images/film/posters/1...
215,Hrozná hnevu,77%,http://img.csfd.cz/files/images/film/posters/1...
212,Experiment,76%,http://img.csfd.cz/files/images/film/posters/0...


In [130]:
muzi.to_json('../data/muzi.json', force_ascii=False, orient='records')

In [131]:
zeny.to_json('../data/zeny.json', force_ascii=False, orient='records')

In [132]:
zena = no_sex_data[['title', 'score', 'img']][no_sex_data.gender == 'F'].sample(20)
zena.to_json('../data/zena.json', force_ascii=False, orient='records')